Мы будем работать с MySQL, так что надо проинсталлировать библиотеку для работы с ним:
pip install mysql-connector-python

In [0]:
# Подключаем библиотеки.
import mysql.connector

import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer

Сперва надо создать соединение с сервером, не забыв указать где находится сервер, с какой базой данных мы собираемся работать, логин и пароль для пользователя.

In [0]:
con=mysql.connector.connect(host="cosyco.ru", database="nplus1", user="mkl_2019", password="M@ster@2019")

Далее создаем курсор для отправки запросов и получения результатов.

In [0]:
cur=con.cursor(dictionary=True)

Выполним несложный запрос при помощи функции execute - выбрать все поля и все записи из таблицы articles. 
SELECT отвечает за выборку данных. * означает, что мы выбираем все поля, но здесь можно написать конкретные поля через запятую. Поля могут содержать имя таблицы, например, articles.id_article. После FROM идут таблицы, из которых делается выборка.

In [0]:
cur.execute("SELECT * FROM articles")

Далее надо выбрать данные из курсора.

In [0]:
res=cur.fetchall()

In [0]:
res

[{'article_author': 'Александр Ершов',
  'article_difficulty': 2.4,
  'article_text': 'Американские\nхимики разработали новое покрытие для\nпищевой тары, благодаря которому жидкости\nвроде меда или кетчупа можно будет\nизвлекать из бутылки полностью, до\nпоследней капли. Описание технологии\nсоздания покрытия опубликовано в журнале\nApplied Materials & Interfaces.\nОсновой\nпокрытия является смесь пчелиного и\nпальмового (карнаубского) воска в\nрастворителе, которую наносят на\nобрабатываемую тару с помощью распылителя.\nПосле высыхания на\nповерхности образуются шероховатая\nмикроструктура, благодаря которой\nплощадь соприкосновения между каплей\nи стенками сводится к минимуму. Наряду\nс гидрофобностью самого воска это дает\nочень малую энергию смачивания, в\nрезультате капли жидкости двигаются\nпо такой поверхности гораздо быстрее,\nчем до обработки.\nПодобный\nподход — использование водоотталкивающего\nвещества, которому дополнительно\nпридается микротекстура — используется\nдля соз

Итак, у нас получилось. Хотя это из-за того, что я уже что-то положил в таблицу.

Теперь посмотрим как можно добавлять записи при помощи INSERT INTO.

In [0]:
cur.execute("INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES ('bla-bla-bla3','bla333-bla-bla','John Doe',2.4), ('bla-bla-bla2','bla-bla-bla2','John Doe','5.4')")

После изменений данных необходимо подтвердить при при помощи функции соединения `connect` или откатить назад при помощи `rollback()`.

In [0]:
con.commit()

Теперь загрузим статьи из NPlus1 и положим их в базу.

In [0]:
delcom=re.compile("<!--.+-->", re.S)

# Класс, хранящий информацию о статье.
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.head=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+'", "difficulty":"'
        res+=self.diff+'", "title":"'+self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.head, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res
    
def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
    t1 = re.split("</time>", re.split("<time", tables)[1])[0]
    art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
    art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head = re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text = BeautifulSoup(art.text, "html5lib")
    art.text = delcom.sub("", beaux_text.get_text() )
    art.text = art.text.replace('\xa0', ' ')

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

def getDayArticles(adr):
    r = requests.get(adr)
    titles = BeautifulSoup(r.text, "html5lib")("article")
    #print(titles)
    addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    #print(addrs)
    articles = []
    for adr in addrs:
        print(adr)
        articles.append(getArticleTextNPlus1(adr))
    return articles

In [0]:
arts=getDayArticles("https://nplus1.ru/news/2016/08/02/")

Посмотрим что вообще загружается.

In [0]:
arts[0].head

'Съедобное покрытие выгонит последнюю каплю сиропа из бутылки'

SQL-запрос можно формировать динамически - это всего лишь строка.

In [0]:
req="""INSERT INTO articles (article_text, article_title, article_author, article_difficulty)
       VALUES ('{}','{}','{}',{})""".format(arts[0].text, arts[0].head, arts[0].author, arts[0].diff)
req

"INSERT INTO articles (article_text, article_title, article_author, article_difficulty)\n       VALUES ('Американские\nхимики разработали новое покрытие для\nпищевой тары, благодаря которому жидкости\nвроде меда или кетчупа можно будет\nизвлекать из бутылки полностью, до\nпоследней капли. Описание технологии\nсоздания покрытия опубликовано в журнале\nApplied Materials & Interfaces.\nОсновой\nпокрытия является смесь пчелиного и\nпальмового (карнаубского) воска в\nрастворителе, которую наносят на\nобрабатываемую тару с помощью распылителя.\nПосле высыхания на\nповерхности образуются шероховатая\nмикроструктура, благодаря которой\nплощадь соприкосновения между каплей\nи стенками сводится к минимуму. Наряду\nс гидрофобностью самого воска это дает\nочень малую энергию смачивания, в\nрезультате капли жидкости двигаются\nпо такой поверхности гораздо быстрее,\nчем до обработки.\nПодобный\nподход — использование водоотталкивающего\nвещества, которому дополнительно\nпридается микротекстура — исп

In [0]:
cur.execute(req)

In [0]:
con.commit()

Положим статьи с сайта в базу данных. Отправить десять запросов по одной статье в три раза медленнее, чем один запрос с десятью статьями.

In [0]:
%%time
for art in arts:
    req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES ('{}','{}','{}',{})".format(art.text.replace("'", "\\\'"), art.head, art.author, art.diff)
    cur.execute(req)
    con.commit()

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 978 ms


In [0]:
%%time
data=[]
for art in arts:
    data.append("('{}','{}','{}',{})".format(art.text.replace("'", "\\\'"), art.head, art.author, art.diff))

req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES "
req+=",".join(data)    
cur.execute(req)
con.commit()

CPU times: user 1.92 ms, sys: 257 µs, total: 2.18 ms
Wall time: 201 ms


Штатно, статьи можно удалять так. Я подозреваю, что у меня где-то произошел клинч - часть статей так и не добавилась, а их уже надо удалять.

In [0]:
cur.execute("DELETE FROM articles")

In [0]:
con.commit()

In [0]:
con.rollback()

Положим теперь не только тексты, но и составляющие их предложения.

Но так как нам нужны еще и идентификаторы текстов, то выберем теперь тексты из базы данных.

По-правильному, нам надо удалить все предложения и занести их заново для всех-всех текстов. Или сохранять где-то номер последнего разложенного на предложения текста. Но это мы не можем сделать, пока все не будут знать что такое `update`. Так что пока делаем плохо - выбираем всех, чей идентификатор больше константы (мы сами запомнили кого мы там в последний раз добавляли).

In [0]:
cur.execute("SELECT id_article, article_text FROM articles where id_article>0")
arts2=cur.fetchall()

for art in arts2:
    sents=art["article_text"].split(".")
    req="INSERT INTO sentences (sent_index, id_article, sent) VALUES "
    req+=",".join(["({},{},'{}')".format(i+1, art["id_article"], s.replace("'", "\\\'")) for i, s in enumerate(sents)])
    cur.execute(req)

In [0]:
con.commit()

Сделаем запрос, который выбирает данные из двух таблиц - первые предложения из статей, сложность которых меньше трех.

In [0]:
cur.execute("SELECT sent, id_sent FROM articles, sentences WHERE article_difficulty<3 AND sent_index=1 AND articles.id_article=sentences.id_article")
res=cur.fetchall()
res

[{'id_sent': 1,
  'sent': 'Американские\nхимики разработали новое покрытие для\nпищевой тары, благодаря которому жидкости\nвроде меда или кетчупа можно будет\nизвлекать из бутылки полностью, до\nпоследней капли'},
 {'id_sent': 14,
  'sent': 'Британские исследователи обнаружили, что удаление небных\nминдалин и аппендикса у женщин связано с повышением вероятности забеременеть'},
 {'id_sent': 28,
  'sent': 'Компания General\nElectric и правительство Никарагуа\nпланируют подключить к интернету вулкан Масая, чтобы\nмониторить его активность в реальном\nвремени'}]

---

In [0]:
cur.execute("SELECT id_article , article_text FROM articles")

In [0]:
arts=cur.fetchall()

In [0]:
morpho=MorphAnalyzer()

In [0]:
%%time
# Кладем слова в словарь.
for art in arts[:10]:
    words=re.findall("[А-Яа-я]+", art['article_text'])
    freqs={}
    for word in words:
        form=morpho.parse(word)[0]
        lemma, pos = form.normal_form, form.tag.POS
        req="SELECT count(*) as cnt FROM vocabulary WHERE lemma='{}' AND partofspeech='{}'". \
            format(str(lemma), str(pos))
        cur.execute(req)
        present=cur.fetchall()
        if present[0]['cnt']==0:
            req="INSERT INTO vocabulary (lemma, partofspeech) VALUES "+\
                "('{}', '{}')".format(str(lemma), str(pos))
            #print(req)
            cur.execute(req)
    con.commit()

CPU times: user 1.49 s, sys: 238 ms, total: 1.72 s
Wall time: 38.1 s


In [0]:
# Кладем слова в обратный индекс
for art in arts[:10]:
    words=re.findall("[А-Яа-я]+", art['article_text'])
    freqs={}
    for word in words:
        form=morpho.parse(word)[0]
        freqs[str(form.normal_form)+"_"+str(form.tag.POS)]=freqs.get(str(form.normal_form)+"_"+str(form.tag.POS), 0)+1
        
    for word, freq in freqs.items():
        lemma, pos = word.split("_")
        req="SELECT id_word FROM vocabulary WHERE lemma='{}' AND partofspeech='{}'". \
            format(str(lemma), str(pos))
        cur.execute(req)
        id_word=cur.fetchall()
        req="INSERT INTO back_index (id_article, id_word, freq) VALUES "+\
                "({}, {}, {})".format(art['id_article'], id_word[0]['id_word'], freq)
        #print(req)
        cur.execute(req)
    con.commit()

In [0]:
# Кладем пары слов в обратный индекс
for art in arts:
    words=re.findall("[А-Яа-я]+", art['article_text'])
    freqs={}
    form=morpho.parse(words[0])[0]
    prev_word=str(form.normal_form)+"_"+str(form.tag.POS)
    for word in words[1:]:
        form=morpho.parse(word)[0]
        cur_word=str(form.normal_form)+"_"+str(form.tag.POS)
        freqs[prev_word+":"+cur_word]=freqs.get(prev_word+":"+cur_word, 0)+1
        prev_word=cur_word
        
    for word, freq in freqs.items():
        prev_word, cur_word = word.split(":")
        lemma, pos = prev_word.split("_")
        req="SELECT id_word FROM vocabulary WHERE lemma='{}' AND partofspeech='{}'". \
            format(lemma, pos)
        cur.execute(req)
        id_word1=cur.fetchall()
        lemma, pos = cur_word.split("_")
        req="SELECT id_word FROM vocabulary WHERE lemma='{}' AND partofspeech='{}'". \
            format(lemma, pos)
        cur.execute(req)
        id_word2=cur.fetchall()
        req="INSERT INTO bigramms (id_article, first, second, freq) VALUES "+\
            "({}, {}, {}, {})".format(art['id_article'], id_word1[0]['id_word'], id_word2[0]['id_word'], freq)
        #print(req)
        cur.execute(req)
    con.commit()

IndexError: list index out of range

In [0]:
con.commit()

In [0]:
# Выбираем частоты биграмм.
req="""SELECT articles.article_title, vocabulary.lemma, voc2.lemma as lemma2, bigramms.freq 
       FROM vocabulary, vocabulary as voc2, bigramms, articles
       WHERE bigramms.freq>1 AND first=vocabulary.id_word AND 
       bigramms.id_article=articles.id_article AND
       second=voc2.id_word
       ORDER BY bigramms.freq DESC
"""

cur.execute(req)
res=cur.fetchall()
for r in res:
    print(r)

{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'lemma': 'на', 'lemma2': 'процент', 'freq': 3}
{'article_title': 'Съедобное покрытие выгонит последнюю каплю сиропа из бутылки', 'lemma': 'с', 'lemma2': 'помощь', 'freq': 3}
{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'lemma': 'процент', 'lemma2': 'процент', 'freq': 2}
{'article_title': 'Съедобное покрытие выгонит последнюю каплю сиропа из бутылки', 'lemma': 'пищевой', 'lemma2': 'тара', 'freq': 2}
{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'lemma': 'после', 'lemma2': 'оба', 'freq': 2}
{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'lemma': 'вероятность', 'lemma2': 'забеременеть', 'freq': 2}
{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'lemma': 'процент', 'lemma2': 'после', 'freq': 2}
{

In [0]:
%%time
# Выбираем частоты отдельных слов.
req="""SELECT bigramms.id_article, bigramms.freq as freq_bi,vocabulary.lemma, back_index.freq as freq1, voc2.lemma as lemma2, back2.freq as freq2, 
       bigramms.freq/(back_index.freq*back2.freq) as mi
       FROM vocabulary, vocabulary as voc2, bigramms, back_index, back_index as back2
       WHERE bigramms.freq>1 AND first=vocabulary.id_word AND 
       second=voc2.id_word AND
       bigramms.first=back_index.id_word AND
       bigramms.second=back2.id_word AND
       bigramms.id_article=back_index.id_article AND
       bigramms.id_article=back2.id_article AND
       (vocabulary.partofspeech='NOUN' OR vocabulary.partofspeech='ADJF' OR vocabulary.partofspeech='VERB') AND 
       (voc2.partofspeech='NOUN' OR voc2.partofspeech='ADJF' OR voc2.partofspeech='VERB')
       ORDER BY mi DESC
"""

cur.execute(req)
res=cur.fetchall()
for r in res:
    print(r)

{'id_article': 44, 'freq_bi': 2, 'lemma': 'пищевой', 'freq1': 3, 'lemma2': 'тара', 'freq2': 3, 'mi': Decimal('0.2222')}
{'id_article': 44, 'freq_bi': 2, 'lemma': 'супергидрофобный', 'freq1': 3, 'lemma2': 'поверхность', 'freq2': 5, 'mi': Decimal('0.1333')}
{'id_article': 45, 'freq_bi': 2, 'lemma': 'процент', 'freq1': 7, 'lemma2': 'процент', 'freq2': 7, 'mi': Decimal('0.0408')}
CPU times: user 0 ns, sys: 3.45 ms, total: 3.45 ms
Wall time: 14.9 ms


In [0]:
%%time
# Выбираем частоты отдельных слов.
#req="""EXPLAIN 
req="""SELECT bigramms.id_article, bigramms.freq as freq_bi,vocabulary.lemma, back_index.freq as freq1, voc2.lemma as lemma2, back2.freq as freq2, 
       bigramms.freq/(back_index.freq*back2.freq) as mi
       FROM vocabulary, 
            (SELECT id_word, lemma FROM vocabulary WHERE 
                    vocabulary.partofspeech='NOUN' OR vocabulary.partofspeech='ADJF' OR vocabulary.partofspeech='VERB'
            ) as voc2, 
            bigramms, back_index, back_index as back2
       WHERE bigramms.freq>1 AND 
       first=vocabulary.id_word AND 
       second=voc2.id_word AND
       bigramms.first=back_index.id_word AND
       bigramms.second=back2.id_word AND
       bigramms.id_article=back_index.id_article AND
       bigramms.id_article=back2.id_article AND
       (vocabulary.partofspeech='NOUN' OR vocabulary.partofspeech='ADJF' OR vocabulary.partofspeech='VERB')
       ORDER BY mi DESC
"""

cur.execute(req)
res=cur.fetchall()
for r in res:
    print(r)

{'id_article': 44, 'freq_bi': 2, 'lemma': 'пищевой', 'freq1': 3, 'lemma2': 'тара', 'freq2': 3, 'mi': Decimal('0.2222')}
{'id_article': 44, 'freq_bi': 2, 'lemma': 'супергидрофобный', 'freq1': 3, 'lemma2': 'поверхность', 'freq2': 5, 'mi': Decimal('0.1333')}
{'id_article': 45, 'freq_bi': 2, 'lemma': 'процент', 'freq1': 7, 'lemma2': 'процент', 'freq2': 7, 'mi': Decimal('0.0408')}
CPU times: user 0 ns, sys: 1.59 ms, total: 1.59 ms
Wall time: 11.1 ms


In [0]:
# Вывести количество слов в статье.

req="""select articles.article_title, count(*) as cnt
from back_index, articles
where back_index.id_article=articles.id_article
group by back_index.id_article"""

cur.execute(req)
res=cur.fetchall()
for r in res:
    print(r)

{'article_title': 'Съедобное покрытие выгонит последнюю каплю сиропа из бутылки', 'cnt': 164}
{'article_title': 'Отсутствие миндалин и аппендикса связали с повышенными шансами забеременеть', 'cnt': 147}
{'article_title': 'Вулкан в Никарагуа подключат к интернету', 'cnt': 100}
{'article_title': 'Физики БАК расскажут о таинственном двухфотонном пике в пятницу', 'cnt': 186}
{'article_title': 'Способность птиц спать на лету доказали энцефалограммой', 'cnt': 203}
{'article_title': 'Найдено новое структурное отличие РНК от ДНК', 'cnt': 210}
{'article_title': 'Швейцария выпустит беспилотники в общее воздушное пространство', 'cnt': 94}
{'article_title': 'Составлена первая гравитационная карта Цереры', 'cnt': 131}
{'article_title': 'Опубликовано видео установки напечатанной детали на конвертоплан', 'cnt': 138}
{'article_title': 'Тайвань открыл центр разработки подводных лодок', 'cnt': 139}


In [0]:
# Хочу красивую табличку.
import pandas as pd

In [0]:
%%time
# Статистика - в какой статье какая часть речи сколько раз встретилась.
req="""select back_index.id_article, vocabulary.partofspeech, count(back_index.id_word) as cnt
from back_index, vocabulary
where back_index.id_word=vocabulary.id_word
group by back_index.id_article, vocabulary.partofspeech"""

cur.execute(req)
res=cur.fetchall()
data=pd.DataFrame(res)
data=data.pivot("id_article", "partofspeech")
data=data.fillna(0)
data


CPU times: user 1.67 ms, sys: 0 ns, total: 1.67 ms
Wall time: 18 ms


In [0]:
%%time
# То же самое, но при помощи join.
req="""select back_index.id_article, vocabulary.partofspeech, count(back_index.id_word) as cnt
from back_index JOIN vocabulary ON back_index.id_word=vocabulary.id_word
group by back_index.id_article, vocabulary.partofspeech"""

cur.execute(req)
res=cur.fetchall()
#data=pd.DataFrame(res)
#data=data.pivot("id_article", "partofspeech")
#data=data.fillna(0)
#data


CPU times: user 623 µs, sys: 4.19 ms, total: 4.82 ms
Wall time: 21.9 ms


In [0]:
# Посчитать относительную частоту частей речи в статьях.
# Делаем в два вложенных запроса и JOIN. Первый запрос возвращает частоту части речи в статье.
# Второй запрос - количество слов в статье. При помощи join объединяем эти данные 
# по номеру статьи, делим одно на другое - получаем относительную частоту.
req="""
SELECT pos_counter.id_article, pos_counter.partofspeech, pos_counter.cnt/art_counter.cnt as rel_cnt
FROM
 (select back_index.id_article, vocabulary.partofspeech, count(back_index.id_word) as cnt
  from back_index, vocabulary
  where back_index.id_word=vocabulary.id_word
  group by back_index.id_article, vocabulary.partofspeech) as pos_counter
 JOIN
 (select id_article, count(*) as cnt
  from back_index
  group by id_article) as art_counter
 ON pos_counter.id_article=art_counter.id_article
 """

cur.execute(req)
res=cur.fetchall()

data=pd.DataFrame(res)
data=data.pivot("id_article", "partofspeech")
data=data.fillna(0)
data


rel_cnt                                                          \
partofspeech    ADJF    ADJS    ADVB    COMP    CONJ    GRND    INFN    INTJ   
id_article                                                                     
44            0.1890  0.0244  0.0915  0.0061  0.0488       0  0.0244       0   
45            0.1769  0.0068  0.0340       0  0.0408       0  0.0340       0   
46            0.0900       0  0.0300       0  0.0300       0  0.1000       0   
47            0.2043  0.0108  0.0376  0.0108  0.0591  0.0054  0.0215       0   
48            0.1675  0.0099  0.0640       0  0.0493  0.0148  0.0246       0   
49            0.2048  0.0143  0.0762  0.0048  0.0476  0.0048  0.0143       0   
50            0.1809  0.0106  0.0426       0  0.0319  0.0106  0.0532       0   
51            0.1145  0.0153  0.0458  0.0076  0.0687       0  0.0382       0   
52            0.1449  0.0145  0.0290       0  0.0435       0  0.0507       0   
53            0.1223  0.0216  0.0288       0  0.0288  0.0072  0.0576  0.0072   

                                                                              \
partofspeech    NOUN    NPRO    NUMR    None    PRCL    PRED    PREP    PRTF   
id_article                                                                     
44            0.3354  0.0183       0       0  0.0305  0.0061  0.0793  0.0244   
45            0.4218  0.0204  0.0068       0  0.0136       0  0.0680  0.0340   
46            0.5100  0.0200       0       0       0  0.0100  0.0900  0.0100   
47            0.3656  0.0215  0.0161  0.0054  0.0161  0.0054  0.0806  0.0323   
48            0.3842  0.0197  0.0049  0.0099  0.0246       0  0.0690  0.0493   
49            0.3810  0.0190  0.0095  0.0048  0.0190       0  0.0810  0.0095   
50            0.4255  0.0213  0.0106       0       0       0  0.0745  0.0319   
51            0.4046  0.0229       0       0  0.0153       0  0.0916  0.0382   
52            0.4203  0.0072  0.0145  0.0145  0.0072  0.0072  0.0870  0.0435   
53            0.4388  0.0144  0.0288  0.0144  0.0216       0  0.0863  0.0216   

                              
partofspeech    PRTS    VERB  
id_article                    
44            0.0183  0.1037  
45            0.0272  0.1156  
46                 0  0.1100  
47            0.0215  0.0860  
48            0.0049  0.1034  
49            0.0190  0.0905  
50            0.0106  0.0957  
51            0.0153  0.1221  
52            0.0145  0.1014  
53                 0  0.1007

In [0]:
# Получить все части речи, которые в статье встретились больше 50 раз.

req="""select back_index.id_article, vocabulary.partofspeech, count(back_index.id_word) as cnt
from back_index, vocabulary
where back_index.id_word=vocabulary.id_word
group by back_index.id_article, vocabulary.partofspeech
having count(back_index.id_word)>50"""


cur.execute(req)
res=cur.fetchall()

data=pd.DataFrame(res)
data

,cnt,id_article,partofspeech
0,55,44,NOUN
1,62,45,NOUN
2,51,46,NOUN
3,68,47,NOUN
4,78,48,NOUN
5,80,49,NOUN
6,53,51,NOUN
7,58,52,NOUN
8,61,53,NOUN
